In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')


forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [3]:
# aoi
from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

✅ AOI loaded: 1 features
   Area: 144217.67 hectares


In [24]:
use_yearly_ee = True
### load the exported gee image
asset_monthly_interpolated = 'projects/remote-sensing-476412/assets/korindo_smooth_monthly'

def annual_col_median(img_col, years):
    def per_year(year):
        start = ee.Date.fromYMD(year, 1, 1)
        # end = start.advance(1, 'year')
        end = ee.Date.fromYMD(year, 12, 31)
        return (
            img_col
            .filterDate(start, end)
            .median()
            .set('year', year)
            .set('system:time_start', start.millis())
        )
    return ee.ImageCollection(years.map(per_year))

monthly_agg = ee.ImageCollection(asset_monthly_interpolated)

# Server-side list of unique years derived from system:time_start
year_list = (
    monthly_agg
        .aggregate_array('system:time_start')         # ee.List of millis
        .map(lambda ts: ee.Date(ts).get('year'))      # convert to year
        .distinct()                                   # keep unique values
        .sort()                                       # optional
)

ee_col_year_median = annual_col_median(monthly_agg, year_list)

year_list_c = year_list.getInfo()
print(year_list_c)

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


In [25]:
ee_col_year_median.first().bandNames().getInfo()

['blue',
 'green',
 'nir',
 'red',
 'redE1',
 'redE2',
 'redE3',
 'redE4',
 'swir1',
 'swir2']

In [26]:
## check fcd data from random points
### wait until all the fcd is processed (exported)!
# ============================================
# STEP 2: Load directly from GCS when needed
# ============================================
# Later, when you need to use the images:
def load_yearly_images_from_gcs(years, gcs_bucket='remote_sensing_saas', base_path='01-korindo/yearly_mosaic_gee'):
    """
    Load yearly images directly from GCS without ingesting to GEE.
    
    Parameters:
    -----------
    years : list[int]
        List of years to load (e.g., [2020, 2021, 2022])
    gcs_bucket : str
        GCS bucket name
    base_path : str
        Base path in bucket
    
    Returns:
    --------
    ee.ImageCollection : Collection of images loaded from GCS
    """
    images = []
    for year in years:
        gcs_path = f'gs://{gcs_bucket}/{base_path}/fcd_{year}.tif'
        img = ee.Image.loadGeoTIFF(gcs_path)
        img = img.set('year', year)
        img = img.set('system:time_start', ee.Date.fromYMD(year, 12, 31).millis())
        images.append(img)
    
    return ee.ImageCollection(images)
fcd_col = load_yearly_images_from_gcs(year_list_c)
fcd_col.first().bandNames().getInfo()

['FCD']

In [27]:
# Filter out empty images (images with 0 bands) before spectralIndices
# This prevents errors when reverse mapping tries to select bands on empty images
print("🔍 Filtering out empty images...")

def add_band_count(img):
    """Add band count as a property"""
    band_count = img.bandNames().size()
    return img.set('has_bands', band_count.gt(0))

ee_col_with_flag = ee_col_year_median.map(add_band_count)
ee_col_year_median = ee_col_with_flag.filter(ee.Filter.eq('has_bands', 1))

print(f"✅ Filtered collection size: {ee_col_year_median.size().getInfo()}")

# Now compute spectral indices
ee_col_indices = ee_col_year_median.spectralIndices(
    index=['EVI', 'GNDVI', 'NDVI','SAVI','NBR'],
    satellite_type='Sentinel',
    G=2.5,
    C1=6.0,
    C2=7.5,
    L=1.0,
    drop=False
)

🔍 Filtering out empty images...
✅ Filtered collection size: 11
🔄 Reverse mapping: Found 10 mappings, 15 bands in result


In [28]:
ee_col_year_median.first().bandNames().getInfo()

['blue',
 'green',
 'nir',
 'red',
 'redE1',
 'redE2',
 'redE3',
 'redE4',
 'swir1',
 'swir2']

In [29]:
ee_col_indices.first().bandNames().getInfo()

['EVI',
 'GNDVI',
 'NDVI',
 'SAVI',
 'NBR',
 'blue',
 'green',
 'red',
 'redE1',
 'redE2',
 'redE3',
 'nir',
 'redE4',
 'swir1',
 'swir2']

In [30]:
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=forestry.config['project']['name'], aoi=aoi_ee.geometry())
wmts.addLayer(ee_col_indices.first(), {'bands': ['NBR'],
   'min': 0,
   'max': 1,
 'gamma': 1.5}, 'nbr_test')

wmts.publish()


INFO:wmts_manager:WMTSManager initialized for project: forestry_carbon_project
INFO:wmts_manager:Added layer: nbr_test
INFO:wmts_manager:Generating map IDs for 1 layers...


Generating GEE Map IDs...


INFO:wmts_manager:AOI processed: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
INFO:wmts_manager:Publishing 1 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: forestry_carbon_project
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'nbr_test': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: forestry_carbon_project_20251201_054025
IN

✅ Centroid calculated successfully with error margin 1
Calculated bbox from coordinates: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
✅ AOI processed successfully:
   - Center: [111.90816469614359, -0.3125187262245936]
   - Area: Unknown
   - BBox: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}


INFO:gee_utils:Refreshing WMTS capabilities...
INFO:gee_utils:WMTS capabilities refreshed successfully
INFO:gee_utils:Updating MapStore WMTS service...
INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 1 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 1
INFO:wmts_manager:WMTS publishing completed: success


{'status': 'success',
 'project_id': 'forestry_carbon_project_20251201_054025',
 'project_name': 'forestry_carbon_project',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 1},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 1 proxy URLs',
  'proxy_urls': {'nbr_test': {'proxy_url': 'http://fastapi:8000/tiles/forestry_carbon_project_20251201_054025/nbr_test/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/4b3ab729e7307eb744236092ce32444b-256a08efe13a35ca56def3a1d5d3facf/tiles/{z}/{x}/{y}',
    'layer_name': 'nbr_test',
    'description': 'NBR_TEST visualization from GEE analysis'}},
  'layers_count': 1},
 'wmts_configuration': {'status': 'success',
  'message': 'WMTS configuration updated successfully',
  'service_id': 'GEE_analysis_WMTS_layers',
  'layers_available': ['forestry_carbon_project_20251201_054025_nbr_test'],
  'layers_count': 

In [31]:
# Get years
years = ee_col_year_median.aggregate_array('year').getInfo()
print(f"Years: {years}")

Years: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


In [32]:
# Get years
years = fcd_col.aggregate_array('year').getInfo()
print(f"Years: {years}")

Years: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


In [33]:
## MERGING TWO IMAGE COLLECTIONS
# Merge with FCD - ensure clean images without swir2 references for xee compatibility
def merge_with_fcd(img):
    """Merge spectral indices with FCD, creating clean images without problematic references"""
    # Get year for matching
    year = img.get('year')
    
    # Select only the spectral indices we need (this creates a new image and breaks computation graph)
    # Force select to ensure we only have the bands we want
    si_selected = img.select(['EVI', 'GNDVI', 'NDVI', 'SAVI', 'NBR'])
    
    # Get matching FCD image - use a simpler approach that avoids ee.Algorithms.If
    # Filter and get first, with a fallback
    fcd_filtered = fcd_col.filter(ee.Filter.eq('year', year))
    
    # Use a conditional that's more explicit
    fcd_exists = fcd_filtered.size()
    fcd_img = ee.Image(
        ee.Algorithms.If(
            fcd_exists.gt(0),
            fcd_filtered.first().select('FCD'),
            ee.Image.constant(0).rename('FCD')
        )
    )
    
    # Merge - ensure both images have bands
    merged = si_selected.addBands(fcd_img)
    
    # Verify the merged image has bands before returning
    # Create a completely new image with explicit band selection to break any computation graph
    final_bands = ['EVI', 'GNDVI', 'NDVI', 'SAVI', 'NBR', 'FCD']
    result = merged.select(final_bands)
    
    # Only set essential properties (avoid copying system:band_names or other problematic properties)
    result = result.set('year', year).set('system:time_start', img.get('system:time_start'))
    
    return result

fcd_col_merged = ee_col_indices.map(merge_with_fcd)

In [34]:
fcd_col_merged.first().bandNames().getInfo()

['EVI', 'GNDVI', 'NDVI', 'SAVI', 'NBR', 'FCD']

In [35]:
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=forestry.config['project']['name'], aoi=aoi_ee.geometry())
wmts.addLayer(fcd_col_merged.first(), {'bands': ['FCD'],
   'min': 0,
   'max': 100,
 'gamma': 1.5}, 'fcd_test')

wmts.addLayer(fcd_col_merged.first(), {'bands': ['NBR'],
   'min': 0,
   'max': 1,
 'gamma': 1.5}, 'nbr_test_fcd')


wmts.publish()


INFO:wmts_manager:WMTSManager initialized for project: forestry_carbon_project
INFO:wmts_manager:Added layer: fcd_test
INFO:wmts_manager:Added layer: nbr_test_fcd
INFO:wmts_manager:Generating map IDs for 2 layers...


Generating GEE Map IDs...


INFO:wmts_manager:AOI processed: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
INFO:wmts_manager:Publishing 2 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: forestry_carbon_project
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'fcd_test': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'nbr_test_fcd': ['tile_url', 'name', 'd

✅ Centroid calculated successfully with error margin 1
Calculated bbox from coordinates: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
✅ AOI processed successfully:
   - Center: [111.90816469614359, -0.3125187262245936]
   - Area: Unknown
   - BBox: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}


ERROR:gee_utils:Error clearing WMTS services: 'list' object has no attribute 'get'
INFO:gee_utils:Refreshing WMTS capabilities...
INFO:gee_utils:WMTS capabilities refreshed successfully
INFO:gee_utils:Updating MapStore WMTS service...
INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 2 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 2
INFO:wmts_manager:WMTS publishing completed: success


{'status': 'success',
 'project_id': 'forestry_carbon_project_20251201_054049',
 'project_name': 'forestry_carbon_project',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 2},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 2 proxy URLs',
  'proxy_urls': {'fcd_test': {'proxy_url': 'http://fastapi:8000/tiles/forestry_carbon_project_20251201_054049/fcd_test/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/2fb595080f88be2f349549995fcba42b-54fa2c832adb1d1572480385eb83b35e/tiles/{z}/{x}/{y}',
    'layer_name': 'fcd_test',
    'description': 'FCD_TEST visualization from GEE analysis'},
   'nbr_test_fcd': {'proxy_url': 'http://fastapi:8000/tiles/forestry_carbon_project_20251201_054049/nbr_test_fcd/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/42bd16c782fbd97edd63cad536ef8386-c62a86c

In [36]:
fcd_col_merged.first().get('system:band_names').getInfo()

['EVI', 'GNDVI', 'NDVI', 'SAVI', 'NBR', 'FCD']

In [37]:
### convert the FCD image collection to the xee
# Now using UTM CRS with meter scale - they match!
import os

## path to the zarr file - GCS
gcs_dir = os.getenv('GCS_ZARR_DIR')

import xee
import numpy as np
import xarray as xr

print("🔄 Converting Earth Engine ImageCollection to xarray Dataset...")
print(f"   CRS: {'EPSG:32749'} (UTM, meters)")
print(f"   Scale: {10}m")
print("   This may take a moment as xee accesses the data from Earth Engine...")

# Reproject aoi_ee to UTM first (for region parameter)
# Use aoi_ee_utm already created in cell 11 (UTM geometry)

from shapely.geometry import box

utm_crs = forestry.config['output_crs']
utm_epsg = int(utm_crs.split(':')[1])
pixel_scale = forestry.config['resolution_satellite']

# bbox = box(*aoi_gpd_utm.total_bounds)
# Get UTM bounds from GeoDataFrame and validate
aoi_gpd_utm = aoi_gpd.to_crs(forestry.config['output_crs'])
bounds_utm = aoi_gpd_utm.total_bounds  # [minx, miny, maxx, maxy]

# Validate bounds are not NaN
if np.any(np.isnan(bounds_utm)):
    raise ValueError(f"Bounds contain NaN values: {bounds_utm}")

xmin, ymin, xmax, ymax = bounds_utm
print(f"UTM Bounds: xmin={xmin:.2f}, ymin={ymin:.2f}, xmax={xmax:.2f}, ymax={ymax:.2f}")
print(f"UTM Bounds span: {(xmax-xmin):.2f}m x {(ymax-ymin):.2f}m")

# Create rectangle geometry in UTM CRS
# rectangle = ee.Geometry.Rectangle(xmin, ymin, xmax, ymax)

## SETUP FOR FCD TO XARRAY
ic = fcd_col_merged

# Method 2: Use transformed geometry directly (more reliable)
# Create UTM geometry from aoi_ee
aoi_ee_utm_geom = aoi_ee.geometry().transform(
    f'EPSG:{utm_epsg}',
    maxError=1
)

ds = xr.open_dataset(
    ic,
    engine='ee',
    crs=utm_crs,
    scale=pixel_scale,
    geometry=aoi_ee_utm_geom # significanly faster to clip with geometry
)
print("✅ Successfully created dataset using transformed geometry")

if 'X' in ds.dims and 'Y' in ds.dims:
    print("Renaming dimensions from X,Y to x,y")
    ds = ds.rename({'X': 'x', 'Y': 'y'})

print(f"\n📦 Dataset created: {type(ds)}")
print(f"   CRS: {utm_crs} (UTM, meters)")
print(f"   Scale: {pixel_scale}m")
print(f"   Dimensions: {dict(ds.dims)}")
print(f"   Data variables: {list(ds.data_vars.keys())}")
print(f"   Coordinates: {list(ds.coords.keys())}")

🔄 Converting Earth Engine ImageCollection to xarray Dataset...
   CRS: EPSG:32749 (UTM, meters)
   Scale: 10m
   This may take a moment as xee accesses the data from Earth Engine...
UTM Bounds: xmin=578614.54, ymin=9949385.44, xmax=623504.21, ymax=9981518.44
UTM Bounds span: 44889.68m x 32133.00m
✅ Successfully created dataset using transformed geometry
Renaming dimensions from X,Y to x,y

📦 Dataset created: <class 'xarray.core.dataset.Dataset'>
   CRS: EPSG:32749 (UTM, meters)
   Scale: 10m
   Dimensions: {'time': 11, 'x': 4489, 'y': 3213}
   Data variables: ['EVI', 'GNDVI', 'NDVI', 'SAVI', 'NBR', 'FCD']
   Coordinates: ['time', 'x', 'y']


/tmp/ipykernel_759/1880987262.py:68: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"   Dimensions: {dict(ds.dims)}")


In [38]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:  (time: 11, x: 4489, y: 3213)
Coordinates:
  * time     (time) datetime64[ns] 88B 2015-01-01 2016-01-01 ... 2025-01-01
  * x        (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y        (y) float64 26kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
Data variables:
    EVI      (time, x, y) float32 635MB ...
    GNDVI    (time, x, y) float32 635MB ...
    NDVI     (time, x, y) float32 635MB ...
    SAVI     (time, x, y) float32 635MB ...
    NBR      (time, x, y) float32 635MB ...
    FCD      (time, x, y) float32 635MB ...
Attributes:
    crs:      EPSG:32749

In [40]:
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr
import os

use_exported_ds = True

# Determine zarr path
zarr_path = os.getenv('GCS_ZARR_DIR', '')
if zarr_path:
    if not zarr_path.startswith('gs://'):
        zarr_path = f"gs://{zarr_path}/ds_resampled_fcd_with_indice_merged.zarr"
    else:
        zarr_path = f"{zarr_path}/ds_resampled_fcd_with_indice_merged.zarr"
    storage = 'gcs'
    # print(f"   Saving to GCS: {zarr_path}")
else:
    zarr_path = os.path.join(os.getcwd(), 'data', 'ds_resampled_fcd_with_indice_merged.zarr')
    storage = 'local'
    print(f"   Saving to local: {zarr_path}")

if use_exported_ds != True:

    # Rename dimensions (X, Y -> x, y)
    # ds_resampled = ds.rename({'X': 'x', 'Y': 'y'}) # no need to do it twice from above
    ds_resampled = ds

    # Save to zarr
    chunk_sizes = {'time': 40, 'x': 1024, 'y': 1024}
    print("   Saving dataset to zarr...")
    save_dataset_efficient_zarr(
        ds_resampled,
        zarr_path,
        chunk_sizes=chunk_sizes,
        compression='lz4',
        compression_level=1,
        overwrite=True,
        storage=storage,
        gee_compatible=True
    )
    print(f"✅ Dataset saved to zarr")

    # Load from zarr to avoid GEE server-side errors
    print(f"\n   Loading from: {zarr_path}")
else:
    ds_resampled = load_dataset_zarr(zarr_path, storage=storage)

print(f"✅ Dataset loaded from zarr")
print(f"   Dataset: {dict(ds_resampled.sizes)}")
print(f"   Variables: {list(ds_resampled.data_vars)}")
print(f"\n✅ ds_resampled ready for tsfresh processing")

INFO:forestry_carbon_arr.utils.zarr_utils:Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/ds_resampled_fcd_with_indice_merged.zarr


📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/ds_resampled_fcd_with_indice_merged.zarr
✅ Dataset loaded: {'time': 11, 'x': 4489, 'y': 3213}
✅ Dataset loaded from zarr
   Dataset: {'time': 11, 'x': 4489, 'y': 3213}
   Variables: ['EVI', 'FCD', 'GNDVI', 'NBR', 'NDVI', 'SAVI']

✅ ds_resampled ready for tsfresh processing


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


In [41]:
ds_resampled

<xarray.Dataset> Size: 4GB
Dimensions:  (time: 11, x: 4489, y: 3213)
Coordinates:
  * time     (time) datetime64[ns] 88B 2015-01-01 2016-01-01 ... 2025-01-01
  * x        (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y        (y) float64 26kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
Data variables:
    EVI      (time, x, y) float32 635MB dask.array<chunksize=(11, 1024, 1024), meta=np.ndarray>
    FCD      (time, x, y) float32 635MB dask.array<chunksize=(11, 1024, 1024), meta=np.ndarray>
    GNDVI    (time, x, y) float32 635MB dask.array<chunksize=(11, 1024, 1024), meta=np.ndarray>
    NBR      (time, x, y) float32 635MB dask.array<chunksize=(11, 1024, 1024), meta=np.ndarray>
    NDVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 1024, 1024), meta=np.ndarray>
    SAVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 1024, 1024), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749